# Week 47

In [2]:
try:
    import fysisk_biokemi
    print("Already installed")
except ImportError:
    %pip install -q "fysisk_biokemi[colab] @ git+https://github.com/au-mbg/fysisk-biokemi.git"

------------------------------------------------------------------------

## The fluorescent protein “mCherry”.

The spectra of many fluorescent proteins can be found at the website:
[www.fpbase.org](https://www.fpbase.org/). Go to FPbase and search for
“mCherry”.

#### (a) Find parameters.

Find the following parameters for the protein

-   Extinction coefficient at absorbance maximum
-   Quantum yield
-   The organism from which it was originally isolated
-   Molecular weight

Save them to seperate variables in the cell below.

In [4]:
... # Your answers here. 

#### (b) Adorbance

What is the absorbance of a 1 µM solution of mCherry at its absorption
maximum?

In [5]:
c = ... # Put the concentration in Molar
l = ... # Path length in cm
A_max = ... # Calculate the adsorbance.
print(f"{A_max = :3.3f}")

#### (c) Extinction coefficient from sequence.

The sequence of the protein is also given. From this determine the
extinction coefficient at 280 nm.

Start by taking the sequence from the website and assigning it to the
variable `sequence` in the cell below.

In [7]:
sequence = ... # Your code here

Now use the sequence to calculate the extinction coefficient

In [9]:
# This is "dictionary" with the extinction coefficients of the relevant 
# amino acid residues. Dictionaries are indexed with 'keys', so you can retrieve
# the value for W as: ext_residue["W"].
ext_residue = {"W": 5500, "Y": 1490, "C": 125}

# Write code to calculate the extinction coefficent
...

Ellipsis

#### (d) What is the concentration of a 1 µM solution of mCherry in mg/mL?

In [11]:
conc_molar = ... # Put the concentration in units of M. 
conc_mg_mL = ... # Calculate the concentration in uts of mg/mL.
print(f"{conc_mg_mL = }")

------------------------------------------------------------------------

The excitation and emission spectra can be downloaded as a csv-file by
clicking the download icon as highlighted below

<figure>
<img
src="https://raw.githubusercontent.com/au-mbg/fysisk-biokemi/refs/heads/main/lessons/figures/week_47/download_spectra.png"
alt="Download spectra" />
<figcaption aria-hidden="true">Download spectra</figcaption>
</figure>

------------------------------------------------------------------------

#### (e) Load the dataset

Use the widget below to load the dataset as a `DataFrame`

In [13]:
from fysisk_biokemi.widgets import DataUploader
from IPython.display import display 
uploader = DataUploader()
uploader.display()

Run the next cell **after** uploading the file

In [14]:
df = uploader.get_dataframe()
display(df)

#### (f) Plot spectra

Make your own plot showing the excitation and emission spectra of
“mCherry” using the above data.

> **Tip**
>
> You don’t have to worry about the `NaN` values in the dataset when
> plotting, matplotlib just skips plotting that line segment.

In [16]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(..., ..., label='Excitation') # Replace ... with your code
... # Replace ... with your code plotting the emission spectrum.
ax.legend()
ax.set_xlabel('Wavelength')
ax.set_ylabel('Extinction & Emission')
plt.show()

#### (g) Stokes shift

What is the Stokes shift of mCherry?

> **Tip**
>
> If you have two arrays `A` and `B` you can find the entry in `A`
> corresponding to the largest value in `B` like this
>
> ``` python
> A_at_B_max = A[np.argmax(B)]
> ```

In [18]:
import numpy as np
lambda_ex = df['wavelength'][np.argmax(df['mCherry ex'])]
lambda_em = df['wavelength'][np.argmax(df['mCherry em'])]
stokes_shift = lambda_em - lambda_ex

print(f"{lambda_ex = :d}")
print(f"{lambda_em = :d}")
print(f"{stokes_shift = :d}")

#### (h) Colors

What colors are the light that correspond to the excitation and emission
maxima respectively?

------------------------------------------------------------------------

## Proteins in blood plasma.

In this exercise we will learn how Python is excellent for handling
datasets with many data points and how it can be used to apply the same
procedure to all the data points at once.

A researcher wants to determine the concentration of two proteins in
blood plasma that is suspected to be involved in development of an
autoimmune disease. 500 patients and 500 healthy individuals were
included in the study and absorbance measurements of the two purified
proteins from all blood plasma samples were measured at 280 nm. The
molecular weight and extinction coefficients of the two proteins are
given in the table below.

| Protein | $M_w$ $[\text{kDa}]$ | $\epsilon$ $[\text{M}^{-1}\text{cm}^{-1}]$ |
|-----------------------|---------------------|-----------------------------|
| 1       | 130                  | 180000                                     |
| 2       | 57                   | 80000                                      |

Properties of the two proteins.

#### (a) Load the dataset

Use the widget to load the dataset as a dataframe from the file
**protein_blood_plasma.xlsx**

In [20]:
from IPython.display import display 
from fysisk_biokemi.widgets import DataUploader
uploader = DataUploader()
uploader.display()

Run this cell **after** having uploaded the file in the cell above.

In [21]:
df = uploader.get_dataframe()
display(df)

#### (b) Calculate concentrations

Calculate the molar concentration of the two proteins in all samples,
the light path for every measurement is 0.1 cm.

Always a good idea to assign known values to variables

In [23]:
protein_1_ext_coeff = 180000
protein_2_ext_coeff = 80000
l = 0.1

> **Tip**
>
> You can set new columns in a `DataFrame` by just assigning to it
>
> ``` python
> df['new_column'] = [1, 2, 3, ..., 42]
> ```
>
> It can also be set as a computation of a property from another row
>
> ``` python
> df['new_column'] = df['current_column'] / 4
> ```

In [24]:
df['protein1_healthy_molar_conc'] = ... # Calculate for concentration in healthy for protein 1.
... # Your code that updates the data frame with the 3 other new columns.
display(df)

#### (c). Concentrations in mg/mL

Add another set of four columns containing the concentrations in mg/mL.

In [26]:
protein_1_mw = 130 * 10**3
protein_2_mw = 57 * 10**3

In [27]:
df['protein1_healthy_conc'] = ...
df['protein1_patient_conc'] = ...
df['protein2_healthy_conc'] = ...
df['protein2_patient_conc'] = ...

names = ['protein1_healthy_conc', 'protein1_patient_conc', 'protein2_healthy_conc', 'protein2_patient_conc']
display(df[names])

#### (d) Mean concentration

Now that we have the concentrations, calculate the concentration in the
four categories.

> **Tip**
>
> When displaying the dataframe above we used indexed it with `names` as
> `df[names]`. We can do the same to compute something over just the
> four rows.
>
> For example the if we have a `DataFrame` called `example_df`, we can
> calculate the mean over the **rows** as:
>
> ``` python
> df[names].mean(axis=0)
> ```
>
> Here `axis=0` means that we apply the operation over the first axis
> which by convention are the rows.

In [29]:
mean = ...
display(mean)

#### (e) Standard deviation

Calculate the standard deviation

> **Tip**
>
> The standard deviation can be calculated using the `.std`-method that
> works in the same way as the `.mean`-method we used above.

In [31]:
std = ...
display(std)

#### (f) Analyze the results

Consider the following questions

-   By comparing healthy individuals with patients, would you expect
    that any of the two proteins would be involved in disease
    development?
-   What additional information does the standard deviation provide
    besides the average value of the protein concentration?